In [38]:
import numpy as np
import pandas as pd

# -> 귀무가설 채택 (0.05<): 가설에 영향을 주지 않는다
# -> 귀무가설 기각 (0.05>): 가설에 영향을 준다.

In [39]:
data=pd.read_csv("C:/Users/82102/Desktop/SW Study/2022_2학년 2학기/전산통계학/데이터/edu_income9.csv")
data.head()

,education,income
0,college,high
1,college,high
2,college,high
3,college,high
4,college,high


In [40]:
# 분할표 만들기
table = pd.crosstab(index=data['education'],columns=data['income'],margins=True,margins_name='Total')
print(table)

income         high  low  midium  Total
education                              
college         255   81     105    441
high-school     111   65      93    269
middle-school    90   86     114    290
Total           456  232     312   1000


In [41]:
from scipy.stats import chi2_contingency
# chi2_contingency(table)
# 1. table: contingency table (위에서 만든 table을 사용하면 됨)
# 2. chi2: test statistic - 검정 통계
# 3. pval: p-value
# 4. dof: degree of freedom - 자유도
# 5. expected: expected frequency - 예상 빈도
chi2,pval,dof,expected = chi2_contingency(table)
print(chi2)
print(pval)
print(dof)
print(expected)
print("----------------------------")
print(chi2_contingency(table)[0])
print(chi2_contingency(table)[1])
print(chi2_contingency(table)[2])
print(chi2_contingency(table)[3])

53.62087164664146
2.2284249810857003e-08
9
[[ 201.096  102.312  137.592  441.   ]
 [ 122.664   62.408   83.928  269.   ]
 [ 132.24    67.28    90.48   290.   ]
 [ 456.     232.     312.    1000.   ]]
----------------------------
53.62087164664146
2.2284249810857003e-08
9
[[ 201.096  102.312  137.592  441.   ]
 [ 122.664   62.408   83.928  269.   ]
 [ 132.24    67.28    90.48   290.   ]
 [ 456.     232.     312.    1000.   ]]


In [42]:
# Q. 다음은 학년에 따른 학교 축제에 참여의 빈도를 보여주는 테이블이다. 알파가 0.05에서 학년별로 비율이 동일한지 테스트해보자!

data = pd.DataFrame({"Grade" : ["G1","G1","G2","G2","G3","G3","G4","G4"], 
                        "Status" : ["Attend","Absent","Attend","Absent","Attend","Absent","Attend","Absent"],
                        "Observed" : [6,48,14,32,13,47,7,33]})
data.head()

,Grade,Status,Observed
0,G1,Attend,6
1,G1,Absent,48
2,G2,Attend,14
3,G2,Absent,32
4,G3,Attend,13


In [43]:
# Create pivot_table -> pd.pivot_table()

# chi2_contingency(table)
# 1. table: contingency table (위에서 만든 table을 사용하면 됨)
# 2. chi2: test statistic - 검정 통계
# 3. pval: p-value
# 4. dof: degree of freedom - 자유도
# 5. expected: expected frequency - 예상 빈도

table = pd.pivot_table(data, values=['Observed'], index=['Status'], columns=['Grade'], 
                        aggfunc=np.sum, margins=True,margins_name='Total')
table

Observed                  
Grade        G1  G2  G3  G4 Total
Status                           
Absent       48  32  47  33   160
Attend        6  14  13   7    40
Total        54  46  60  40   200

In [44]:
# Chi-square Test
print("Test Statistic: ", chi2_contingency(table)[0])
print("P-value: ",chi2_contingency(table)[1])

print("---------------------------------------------------")

chi2,pval,dof,expected = chi2_contingency(table)
print(chi2)
print(pval)
print(dof) # 자유도
print(expected)

Test Statistic:  6.057518115942029
P-value:  0.6407889520854085
---------------------------------------------------
6.057518115942029
0.6407889520854085
8
[[ 43.2  36.8  48.   32.  160. ]
 [ 10.8   9.2  12.    8.   40. ]
 [ 54.   46.   60.   40.  200. ]]


In [45]:
# Fisher's Exact Test - 피셔정확검정

data = pd.DataFrame({"AB" : ["A","A","B","B",], 
                        "G12" : ["G1","G2","G1","G2"],
                        "Observed" : [1,4,8,5]})
data.head()

,AB,G12,Observed
0,A,G1,1
1,A,G2,4
2,B,G1,8
3,B,G2,5


In [46]:
table = pd.pivot_table(data, values=['Observed'], index=['G12'], columns=['AB'], 
                        aggfunc=np.sum, margins=False)
table

Observed   
AB         A  B
G12            
G1         1  8
G2         4  5

In [47]:
# Observed Probability
import math

def Ob_Pro(table):
    n,p = table.shape
    out1 = 1
    out2 = 1
    tot_n = 0
    
    for i in range(n):
        tot_n += np.sum(table.iloc[i,:])
        out1 *= math.factorial(np.sum(table.iloc[i,:]))
        for j in range(p):
            out2 *= math.factorial(table.iloc[i,j])
    
    out2 *= math.factorial(tot_n)
    for j in range(p):
        out1 *= math.factorial(np.sum(table.iloc[:,j]))
    
    result = out1/out2
    
    return result

print("Observed Probability : ", np.round(Ob_Pro(table),4))

Observed Probability :  0.1324


In [48]:
# Compute P-value

from scipy.stats import fisher_exact

_,pval = fisher_exact(table,alternative='two-sided')

print("P-Value : ",np.round(pval,4))

if (pval > 0.05):
    print("귀무가설 채택 - 가설이 영향 없다")
else:
    print("귀무가설 기각 - 가설이 영향 있다")

P-Value :  0.2941
귀무가설 채택 - 가설이 영향 없다
